In [44]:
import tensorflow as tf
tf.reset_default_graph() # 重置计算图

In [45]:
with tf.variable_scope("zhang"):
    with tf.variable_scope("kai"):
        v = tf.get_variable("b",[1])  # 可以应用于首次定义该变量的时候, 
        assert v.name == "zhang/kai/b:0"

In [46]:
with tf.variable_scope("zhang") as tf1:
    pass

In [47]:
# 安全的方式 进入一个命名空间
with tf.variable_scope(tf1, auxiliary_name_scope = False,reuse = False) as tf2:
    with tf.name_scope(tf1.original_name_scope):
        w = tf.get_variable("b",[1])
        assert w.name == "zhang/b:0"

In [63]:
# 共享一个变量
tf.reset_default_graph()
def foo():
    with tf.variable_scope("zhang", reuse = tf.AUTO_REUSE): # this must be tf.AUTO_REUSE
        v = tf.get_variable("v",[1])
    return v
v = foo()
w = foo()
assert v == w

In [71]:
# one way to reuse variable
tf.reset_default_graph()
with tf.variable_scope("zhang"):
    v = tf.get_variable("v",[1])
with tf.variable_scope("zhang",reuse = True): # reuse
    v1 = tf.get_variable("v",[1])

In [70]:
tf.reset_default_graph()
# 在variable_scope 中重新使用变量
with tf.variable_scope("zhang") as scope:
    v = tf.get_variable("v",[1])
    scope.reuse_variables()
    v1 = tf.get_variable("v",[1])
    assert v == v1

In [114]:
# 多线程共享变量
tf.reset_default_graph()
from threading import Thread
    
main_thread_scope = tf.get_variable_scope()
print(main_thread_scope.name)


def thread_target_fn(captured_scope):
    with tf.variable_scope(captured_scope):
        v = tf.get_variable("zhang",[1])
    return v

class MyThread(Thread):
    def __init__(self,func,args=()):
        super(MyThread,self).__init__()
        self.func = func
        self.args = args
    def run(self):
        self.result = self.func(*self.args)
        
    def get_result(self):
        Thread.join(self) # 要加上，不然因为主线程快速
        try:
            return self.result  # 如果子线程不使用join方法，此处可能会报没有self.result的错误
        except Exception:
            return None
        
for i in range(10):
    t = MyThread(thread_target_fn,(main_thread_scope,))
    t.start()
    print(t.get_result())


<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'zhang:0' shape=(1,) dtype=float32_ref>
